In [1]:
import os 
import pandas as pd

def get_min_max(path):
    df_list = []
    for dir in os.listdir(path):
        for subdir in os.listdir(os.path.join(path,dir)):
            load_path = os.path.join(path, dir, subdir)
            df = pd.read_csv(load_path)
            df_list.append(df)
    df = pd.concat((df_list[0], df_list[1], df_list[2], df_list[3], df_list[4], df_list[5]))
    return df
    


def create_plateau(n, df):
    plateau_counts = []
    # max_val = float(df.max())
    # print("Max: ",max_val)
    # min_val = float(df.min())
    # print("Min: ",min_val)
    max_val = 1157.6
    min_val = 43.2
    plateau_list = []
    plateau_size = (max_val-min_val)/n
    print("Plateua Size: ",plateau_size)
    current_plateau = min_val
    for i in range(0,n):
        new_plateau = current_plateau + plateau_size
        print(str(current_plateau) + " --- " + str(new_plateau))
        df_filter = int(df[(df["BUS"] > current_plateau) & (df["BUS"] <= new_plateau)].count()[0])
        current_plateau = new_plateau
        print(df_filter)
        plateau_counts.append(df_filter)
    return plateau_counts

def only_max(df, tolerance):
    max_val = float(df.max())
    plateau_count = df[df["BUS"] >= (1157.6-tolerance)].count()[0]
    return plateau_count


def load_data(path, plateau_size=None):
    no_dash_list = []
    dash_list = []

    for dir in os.listdir(path):
        for subdir in os.listdir(os.path.join(path,dir)):
            load_path = os.path.join(path, dir, subdir)
            df = pd.read_csv(load_path)
            df = df[["BUS"]]
            if plateau_size is None:
                plateau_counts = only_max(df,tolerance=10)
            else:
                plateau_counts = create_plateau(plateau_size, df)
            if "no" in subdir:
                no_dash_list.append(plateau_counts)
            else:
                dash_list.append(plateau_counts)
    NoDash = pd.DataFrame(no_dash_list)
    NoDash["Dash"] = "no"
    Dash = pd.DataFrame(dash_list)
    Dash["Dash"] = "yes"
    df = pd.concat((NoDash,Dash))
    return df.sample(frac=1).reset_index(drop=True)
    

In [4]:
df = get_min_max("Printer3D")
df.dropna(inplace=True)
print(df.BUS.max())
print(df.BUS.min())

1157.6
43.2


In [9]:
# if platue_size is not assigned like: load_data(path = "Printer3D")
# then there are no plateaus. Instead there is a tolerance of the maximum value of the current
# and everything above this tolerance is counted 
# Example: max value of current: 1000mA
# Tolerance: 100mA
# Result: Every current above 900mA is counted

# df = load_data(path = "Printer3D")

# with plateaus: assign desired number of plateaus
df = load_data(path = "Printer3D", plateau_size=20)
df

Plateua Size:  55.71999999999999
43.2 --- 98.91999999999999
1614
98.91999999999999 --- 154.64
15740
154.64 --- 210.35999999999999
15064
210.35999999999999 --- 266.08
30308
266.08 --- 321.79999999999995
42646
321.79999999999995 --- 377.5199999999999
33511
377.5199999999999 --- 433.2399999999999
1
433.2399999999999 --- 488.95999999999987
34
488.95999999999987 --- 544.6799999999998
13
544.6799999999998 --- 600.3999999999999
0
600.3999999999999 --- 656.1199999999999
2
656.1199999999999 --- 711.8399999999999
0
711.8399999999999 --- 767.56
4
767.56 --- 823.28
0
823.28 --- 879.0
2437
879.0 --- 934.72
27600
934.72 --- 990.44
16799
990.44 --- 1046.16
24765
1046.16 --- 1101.88
10685
1101.88 --- 1157.6000000000001
11936
Plateua Size:  55.71999999999999
43.2 --- 98.91999999999999
2497
98.91999999999999 --- 154.64
16402
154.64 --- 210.35999999999999
16964
210.35999999999999 --- 266.08
35108
266.08 --- 321.79999999999995
38892
321.79999999999995 --- 377.5199999999999
39606
377.5199999999999 --- 433.

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,Dash
0,1385,15388,16647,37802,37433,40673,0,15,16,0,...,0,0,0,37,6477,5232,10224,11706,11915,no
1,2549,14830,16043,37710,35089,43697,1,12,12,0,...,0,1,2,19,5378,5284,10824,10018,12117,no
2,1614,15740,15064,30308,42646,33511,1,34,13,0,...,0,4,0,2437,27600,16799,24765,10685,11936,yes
3,2048,15963,17020,37681,33973,42199,2,11,12,1,...,1,1,0,375,6428,4309,10946,12027,13945,no
4,2497,16402,16964,35108,38892,39606,0,9,10,0,...,0,0,0,56,2225,5169,10764,12447,11085,yes
5,1561,15717,16738,37894,39573,40068,1,11,12,3,...,0,1,1,147,5952,4771,11023,10722,12729,yes


In [10]:
# store dataset for train/test model
df.to_csv("3d_printer_plateau.csv", index=False)

In [11]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import cross_val_score 
y = df[["Dash"]]
X = df.drop("Dash", axis=1)
dt = DecisionTreeClassifier()
# don't want to extract testset, instead using Cross Validation
cv_results = cross_val_score(dt, X, y, cv=3)
print(cv_results)

[0.5 1.  0.5]


Clean data_dash0 because of late start

In [256]:
df_1 = pd.read_csv("Printer3D/Dash/data_dash1.csv")
# compare size with another dataset
print(df_1.shape[0])
df = pd.read_csv("data_dash0.csv")
# delete the starting procedure
df = df.loc[40000:]
# store it without index
df.to_csv("Printer3D/Dash/data_dash0_gereinigt.csv", index=False)
# validate size of new datas
df


191235


,Time,MCU,BUS
40000,120324.0,5.5,1022.9
40001,120327.0,5.5,1029.9
40002,120330.0,5.6,1032.6
40003,120333.0,6.0,1033.1
40004,120336.0,5.2,1028.5
...,...,...,...
233155,695864.0,5.8,115.4
233156,695867.0,5.8,117.0
233157,695870.0,6.3,115.8
233158,695873.0,5.8,116.2
